modbus parse to json

# data

In [191]:
data_1 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 5, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 147, 161, 165, 176, 175, 174, 175, 31, 34, 36, 43, 43, 34, 0, 0, 16880, 0, 16231, 7282, 17008, 0, 15921, 50973, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16601, 39322, 16512, 0, 32704, 0, 32704, 0, 0, 0, 0, 0, 0, 0, 16055, 21846, 17740, 52019, 0, 0, 17065, 0, 15860, 29127, 17209, 32768, 17191, 41444]
data_2 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 30, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 81, 80, 81, 82, 112, 152, 149, 31, 34, 36, 43, 43, 34, 0, 0, 0, 0, 16231, 7282, 0, 0, 15921, 50973, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32704, 0, 32704, 0, 32704, 0, 32704, 0, 0, 0, 16057, 12136, 17740, 52019, 0, 0, 0, 0, 15853, 2426, 17133, 39322, 17170, 58766]
data_3 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 35, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 81, 80, 80, 107, 146, 142, 31, 34, 37, 43, 42, 34, 0, 0, 0, 0, 16226, 19418, 0, 0, 15918, 4855, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32704, 0, 32704, 0, 32704, 0, 32704, 0, 0, 0, 16038, 43690, 17740, 52019, 0, 0, 0, 0, 15860, 29127, 17120, 52429, 17169, 16450]
len(data_1)

104

# data extraction

In [196]:
import csv
import math
import struct

In [223]:
def adjust_offset():
    """
    Read and adjust offset value
    Known bug: children offset < parent offset will not work
    
    Send:
        float: next offset
    Yields:
        float: absolute offset relative to position 0
    """
    cur = parent = prev = offset = 0
    while True:
        inc = cur - prev
        if inc < 0:
            parent = prev
        #print('g cur: {}, inc: {}, par: {}'.format(cur, inc, parent))
        prev = cur
        if cur < parent:
            print('under parent')
            cur = yield parent + cur
        else:
            cur = yield cur
            parent = cur

# adjust offset example:
rel_offsets = [0, 2, 4, 6, 0.0, 0.1, 0.2, 0.3, 0.4, 1.1, 8, 10, 0,   4,  8]
abs_offsets = [0, 2, 4, 6, 6.0, 6.1, 6.2, 6.3, 6.4, 7.1, 8, 10, 10, 14, 18]
ao_test = adjust_offset()
next(ao_test)
for i in range(len(rel_offsets)):
    adjusted = ao_test.send(float(rel_offsets[i]))
    print(abs_offsets[i], adjusted)
    #assert adjusted == abs_offsets[i], 'adjust offset'


def unpack_bool(data):
    """Get the boolean data
    
    Args:
        data: integer
        position: location of bool to extract
    Return:
        list of bool
    """
    return [bool(int(i)) for i in '{:016b}'.format(data)]

def decimals_to_float32(int1, int2):
    """Convert to float
    
    Note:
        Orders in big endian
    Args:
        int1, int2: two interger to combine
    Return:
        number in float
    """
    f = int('{:016b}{:016b}'.format(int1, int2), 2)
    return struct.unpack('f', struct.pack('I', f))[0]

def unpack_real(data, offset):
    """Get data and convert to float
    
    Args:
        data: List of data
        offset: starting address
    Return:
        number in float
    """
    return decimals_to_float32(data[offset//2], data[offset//2+1])

def get_val(data, position, data_type):
    """
    arg:
        data, position, data type
    return:
        coorsponding value
    """
    if data_type == 'Int':
        return data[int(position)//2]
    elif data_type == 'Bool':
        bool_pos = round(10*(position-math.floor(position)))
        if math.floor(position)%2 == 1: # add position for boolean at offset in odd positions
            bool_pos += 8
        return unpack_bool(data[math.floor(position//2)])[bool_pos]
    elif data_type == 'Real':
        return unpack_real(data, int(position))
    else:
        return '-other-'

0 0.0
2 2.0
4 4.0
6 6.0
under parent
6.0 6.0
under parent
6.1 6.1
under parent
6.2 6.2
under parent
6.3 6.3
under parent
6.4 6.4
under parent
7.1 7.1
8 8.0
10 10.0
under parent
10 10.0
under parent
14 14.0
under parent
18 18.0
22 12.0
26 16.0


In [198]:
# milestone 1
with open('15103EZM_DB201_1.csv', 'r') as f:
    reader = csv.reader(f, doublequote=True, quoting=csv.QUOTE_ALL, escapechar='\\')
    
    print_format = '{:12.12} {:9.9}|abs_pos: {:4.4}|Value: {:5}|ex Value: {}'
    ao = adjust_offset()
    next(ao)
    rel_offset = abs_offset = 0
    
    for row in reader:
        '''0: name, 1: type, 2: offset'''
        rel_offset = row[2]
        abs_offset = ao.send(float(rel_offset))
        ex_value = get_val(data_1, abs_offset, row[1])

        try:
            value = data_1[int(rel_offset)//2]
        except:
            #print('error')
            value = 'Error'
        print(print_format.format(row[0], '({})'.format(row[1]), rel_offset, value, ex_value))

Job_Num      (Int)    |abs_pos: 0   |Value:     0|ex Value: 0
MC_st        (Int)    |abs_pos: 2   |Value:     3|ex Value: 3
EX1_status   (Int)    |abs_pos: 4   |Value:     0|ex Value: 0
EX2_status   (Int)    |abs_pos: 6   |Value:     0|ex Value: 0
LSP_in_prese (Int)    |abs_pos: 8   |Value:     0|ex Value: 0
Spark_H_Acc  (Int)    |abs_pos: 10  |Value:     0|ex Value: 0
Spark_C_Acc  (Int)    |abs_pos: 12  |Value:     0|ex Value: 0
pb           (Struct) |abs_pos: 14  |Value:     0|ex Value: -other-
start        (Bool)   |abs_pos: 0   |Value:     0|ex Value: False
stop         (Bool)   |abs_pos: 0.1 |Value: Error|ex Value: False
inc_ex1      (Bool)   |abs_pos: 0.2 |Value: Error|ex Value: False
dec_ex1      (Bool)   |abs_pos: 0.3 |Value: Error|ex Value: False
inc_ex2      (Bool)   |abs_pos: 0.4 |Value: Error|ex Value: False
dec_ex2      (Bool)   |abs_pos: 0.5 |Value: Error|ex Value: False
inc_lsp      (Bool)   |abs_pos: 0.6 |Value: Error|ex Value: False
dec_lsp      (Bool)   |abs_pos: 0.7 

# helper function test area

In [13]:
import struct

def decimals_to_float32(int1, int2):
    '''Convert to float
    
    Note:
        Orders in big endian
    Args:
        int1, int2: two interger to combine
    Return:
        number in float
    '''
    f = int('{:016b}{:016b}'.format(int1, int2), 2)
    return struct.unpack('f', struct.pack('I', f))[0]

def unpack_real(data, offset):
    '''Get data and convert to float
    
    Args:
        data: List of data
        offset: starting address
    Return:
        number in float
    '''
    return decimals_to_float32(data[offset//2], data[offset//2+1])

print(unpack_real(data_1, 96))
print(unpack_real(data_1, 104))
print(unpack_real(data_1, 108))
print(unpack_real(data_1, 200))
print(unpack_real(data_1, 204))

0.9027777910232544
0.173611119389534
9859.099609375
185.5
167.63238525390625


In [58]:

def unpack_bool(data):
    '''Get the boolean data
    
    Args:
        data: integer
        position: location of bool to extract
    Return:
        list of bool
    '''
    return [bool(int(i)) for i in '{:016b}'.format(data)]

for i in range(16):
    print(unpack_bool(32768)[i])
    


True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


# scrape area

In [37]:
#st = 'abc\x69\xe6\x03'
#bytes(st, 'latin1') # Note that 'ascii' encoding gives error for some values.
st.encode('latin1')

b'abci\xe6\x03'